In [1]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 MB 11.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 15.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# test catboost

import numpy
from catboost import CatBoostRegressor

dataset = numpy.array([[1,4,5,6],[4,5,6,7],[30,40,50,60],[20,15,85,60]])
train_labels = [1.2,3.4,9.5,24.5]
model = CatBoostRegressor(learning_rate=1, depth=6, loss_function='RMSE')
fit_model = model.fit(dataset, train_labels)

print(fit_model.get_params())

0:	learn: 6.8953900	total: 46.6ms	remaining: 46.5s
1:	learn: 4.8590818	total: 47.2ms	remaining: 23.6s
2:	learn: 3.6271477	total: 47.3ms	remaining: 15.7s
3:	learn: 2.7203608	total: 47.7ms	remaining: 11.9s
4:	learn: 2.0402706	total: 47.9ms	remaining: 9.54s
5:	learn: 1.5302029	total: 48.2ms	remaining: 7.99s
6:	learn: 1.1476522	total: 48.6ms	remaining: 6.9s
7:	learn: 0.8607391	total: 48.8ms	remaining: 6.05s
8:	learn: 0.6455544	total: 49ms	remaining: 5.4s
9:	learn: 0.4841658	total: 49.2ms	remaining: 4.87s
10:	learn: 0.3631243	total: 49.3ms	remaining: 4.43s
11:	learn: 0.2723432	total: 49.4ms	remaining: 4.06s
12:	learn: 0.2042574	total: 49.5ms	remaining: 3.76s
13:	learn: 0.1531931	total: 49.6ms	remaining: 3.5s
14:	learn: 0.1148948	total: 49.8ms	remaining: 3.27s
15:	learn: 0.0861711	total: 50ms	remaining: 3.08s
16:	learn: 0.0646283	total: 50.2ms	remaining: 2.9s
17:	learn: 0.0484712	total: 50.3ms	remaining: 2.74s
18:	learn: 0.0363534	total: 50.4ms	remaining: 2.6s
19:	learn: 0.0272651	total: 50.

In [3]:
def get_labels(directory, metadata_csv):
    metadata = pd.read_csv(metadata_csv, index_col=0, header=None)

    labels = {0: 'sham', 1: 'ctbs', 2: 'itbs'}
    data = []

    for filename in os.listdir(directory):
        # note that the s can be upper or lower case and that the letter b can be behind the session number
        match = re.match(r'TMS-EEG-H_(\d+)_(S|s)(\w+)(b?)_(rsEEG|spTEP)_(pre|post)-epo.fif', filename)
        if match:
            patient_id, _, session, _, eeg_type, pre_post = match.groups()
            session = int(session.rstrip('b'))

            # Get the procedure for the session from the metadata
            procedure = labels[metadata.loc[f'H{patient_id}'][session]]

            data.append([filename, procedure, patient_id, eeg_type, pre_post])

    df = pd.DataFrame(data, columns=['filename', 'procedure', 'patient_id', 'eeg_type', 'pre_post'])
    return df

get_labels("dataset-cleaned", "Randomisatielijst.csv")

,filename,procedure,patient_id,eeg_type,pre_post
0,TMS-EEG-H_02_S1b_rsEEG_post-epo.fif,itbs,02,rsEEG,post
1,TMS-EEG-H_02_S1b_spTEP_pre-epo.fif,itbs,02,spTEP,pre
2,TMS-EEG-H_02_S1b_rsEEG_pre-epo.fif,itbs,02,rsEEG,pre
3,TMS-EEG-H_02_S1b_spTEP_post-epo.fif,itbs,02,spTEP,post


In [35]:
def get_data(directory, filenames):
    """Returns a df containing all rows from the given feature files."""
    data = []
    for filename in filenames:
        df = pd.read_csv(os.path.join(directory, filename), header=[0,1])
        data.append(df)
    df = pd.concat(data)
    return df

def get_labels(label_name, label_df, filenames):
    """Returns a df column containg the labels for the given filenames. This is achieved by repeating the respective label for each row in the feature file."""
    labels = []
    print(filenames)
    for filename in filenames:
        df = pd.read_csv(os.path.join("features", filename), header=[0,1])
        rows = df.shape[0]
        filename = filename.split(".")[0]
        label = label_df[label_df['filename'] == filename][label_name].values[0]
        print(f'rows: {rows}, label: {label}')
        # add label * rows times
        labels.extend([label] * rows)
    df = pd.DataFrame(labels, columns=[label_name])
    return df

files = []
requirements = ["rsEEG"]
for filename in os.listdir("features"):
    if all(x in filename for x in requirements):
        files.append(filename)

data_df = get_data("features", files)

label_df = pd.read_csv("labels.csv")
label_df = get_labels("timing", label_df, files)

label_df

['TMS-EEG-H_02_S1b_rsEEG_post-epo.csv', 'TMS-EEG-H_02_S1b_rsEEG_pre-epo.csv']
rows: 296, label: 1
rows: 317, label: 0


,timing
0,1
1,1
2,1
3,1
4,1
...,...
608,0
609,0
610,0
611,0


In [37]:
import numpy as np

from catboost import CatBoostClassifier, Pool

# initialize data
train_data = data_df

train_labels = label_df

test_data = catboost_pool = Pool(train_data,
                                 train_labels)

model = CatBoostClassifier(iterations=5,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(train_data, train_labels)
# make the prediction using the resulting model
preds_class = model.predict(test_data)
preds_proba = model.predict_proba(test_data)
print("class = ", preds_class)
print("proba = ", preds_proba)


0:	learn: 0.1804576	total: 70.4ms	remaining: 282ms
1:	learn: 0.0794103	total: 129ms	remaining: 194ms
2:	learn: 0.0612350	total: 170ms	remaining: 113ms
3:	learn: 0.0387861	total: 209ms	remaining: 52.4ms
4:	learn: 0.0246619	total: 245ms	remaining: 0us
class =  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0